# Predicting Network Anomalies: Model Evaluation

We use the data and the labels generated in step 1 to build and evaluate the performance of multiple models to predict network degradation as it is identified in the labels. 

## Imports

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd


In [ ]:
clean_data = pd.read_csv("clean_labeled.csv")

labels = clean_data["predictions"]
X = clean_data.drop(columns=['basic_ema_anomaly', 'dspot_anomaly', 'predictions'])

,Unnamed: 0,hostname,date,ping_jitter,ping_latency,ping_low,ping_high,day
0,0,9cd30bf,2025-04-20 12:00:30,1.824,15.667,12.622,17.349,2025-04-20
1,1,b340432,2025-04-21 00:00:30,5.070,17.431,8.240,18.138,2025-04-21
2,2,b340432,2025-04-21 12:00:30,5.070,17.431,8.240,18.138,2025-04-21
3,4,9dc32f2,2025-04-22 12:00:30,8.191,11.906,8.095,21.929,2025-04-22
4,5,33fe84e,2025-04-23 03:00:24,1.036,13.099,12.911,14.710,2025-04-23
...,...,...,...,...,...,...,...,...
28844,44764,b340432,2025-07-31 21:00:37,4.611,12.877,9.498,18.723,2025-07-31
28845,44765,a2e0486,2025-07-31 21:00:37,0.795,17.783,16.430,18.231,2025-07-31
28846,44766,953d46d,2025-07-31 21:00:38,2.457,13.381,12.142,16.663,2025-07-31
28847,44767,33fe84e,2025-07-31 21:00:42,3.244,11.808,9.115,14.302,2025-07-31


In [38]:
from datetime import datetime, timedelta

hostdf = X[X['hostname'] == 'b340432'].copy()
hostdf["date"] = pd.to_datetime(hostdf["date"])
hostdf['time_diff'] = hostdf['date'] - (hostdf['date'].shift(1))
mask = (hostdf['time_diff'] > timedelta(hours=2)) & (hostdf['time_diff'] <= timedelta(hours=3, minutes=15))
filtered_results = hostdf[mask]['time_diff'].value_counts().head(20)
filtered_results

# 458/887 have interval of ~3 hours
# most others have really short interval ~10 seconds

time_diff
0 days 03:00:00    36
0 days 02:59:54    24
0 days 02:59:56    17
0 days 02:59:53    15
0 days 02:59:57    15
0 days 02:59:51    15
0 days 02:59:37    14
0 days 02:59:58    14
0 days 02:59:39    14
0 days 02:59:47    13
0 days 02:59:41    13
0 days 02:59:45    13
0 days 02:59:44    12
0 days 02:59:35    12
0 days 02:59:55    12
0 days 02:59:40    12
0 days 02:59:50    12
0 days 03:00:03    11
0 days 02:59:46    11
0 days 02:59:49    10
Name: count, dtype: int64